In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Markdown

### A text-summarizer for websites using Ollama

In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given URL using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator = "\n", strip = True)

In [4]:
#system-prompt and user-prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring texts that might be navigation related. \
Respond in markdown."
def user_prompt_for(website): 
    user_prompt = f"You are looking at a website titled {website.title}."
    user_prompt += "The contents of the website is as follows \
    Please provide a short summary of the website in markdown. \
    If it includes news or announcements, summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [5]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [6]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [7]:
def summarize(url):
    website = Website(url)
    payload = {"messages": messages_for(website), "model": MODEL, "stream": False}
    response = requests.post(OLLAMA_API, headers = HEADERS, json = payload)
    print(response.json()["message"]["content"])

In [8]:
summarize("https://www.theodinproject.com/")

**Summary of the Website**

The website is titled "Your Career in Web Development Starts Here | The Odin Project" and appears to be a free, open-source online learning platform for web development.

### Curriculum Overview

* Offers a full stack curriculum that includes HTML, CSS, JavaScript, React, NodeJS, Ruby on Rails, and Ruby.
* Curated online tutorials, blogs, and courses are presented in a logical order.
* Students can build dozens of portfolio-worthy projects along the way.

### Community Features

* Friendly community of beginner and experienced developers.
* Forum for discussion and help with learning.

### Success Stories

The website showcases success stories from students who have completed the program, including:

* Rob Pando: "TOP was a perfect match for my learning style."
* Cody Loyd: "I was offered a job as a front-end developer after completing the program."
* Andrej Dragojević: "An incredible self-paced curriculum that consisted of the best resources for learning pr